In [1]:
## Definindo soluções de exemplo
from ga4stpg.graph import UGraph

common_edges =[
    (3, 5),
    (8, 17),
    (14, 17),
    (14, 30),
    (25, 30),
    (27, 28),
    (11, 16)
]

red_edges = [
    (1, 4),
    (2, 4),
    (3, 4),
    (5, 6),
    (6, 7),
    (7, 8),
    (17, 18),
    (18, 20),
    (20, 21), 
    (21, 23),
    (14, 13),
    (13, 12),
    (12, 11),
    (30, 29),
    (29, 24),
    (24, 28)
]

blue_edges = [
    (1, 2),
    (2, 3),
    (5, 9),
    (9, 10),
    (10, 8),
    (17, 19),
    (19, 20),
    (20, 22),
    (22, 23),
    (25, 26),
    (26, 24),
    (24, 27),
    (14, 15),
    (15, 16),
]

In [3]:
red = UGraph()
blue = UGraph()

for v, u in common_edges:
    red.add_edge(v, u)
    blue.add_edge(v, u)

for v, u in red_edges:
    red.add_edge(v, u)

for v, u in blue_edges:
    blue.add_edge(v, u)

In [4]:
from ga4stpg.graph.util import has_cycle, how_many_components

print(has_cycle(red), how_many_components(red))
print(has_cycle(blue), how_many_components(blue))

False 1
False 1


1. percorrer todas as arestas das componentes conexas
2. classificar essas arestas
3. encontrar os vértices comuns
4. determinar as partições

In [5]:
g_star = UGraph()

for v, u in red.gen_undirect_edges():
    if not blue.has_edge(v, u):
        g_star.add_edge(v, u)
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        g_star.add_edge(v, u)

print(has_cycle(g_star), how_many_components(g_star))

True 5


In [12]:
def display(a, b, back_edge=False, msg=''):
    if back_edge:
        print(a, b, msg, "<back edge>")
    else:
        print(a, b, msg)

In [26]:
vertices = set(g_star.vertices)

def search(graph, start):
    
    stack = [start]
    stacked = set(stack)
    parent = {start : None}

    count = 0
    while stack:
        v = stack.pop()
        stacked.remove(v)
        for w in graph.adjacent_to(v):
            count += 1
            if w not in parent:
                txt = 'red' if red.has_edge(v, w) else 'blue'
                parent[w] = v
                stack.append(w)
                stacked.add(w)
                display(v, w, msg=txt)
            elif w in stacked and w != parent[v]:
                txt = 'red' if red.has_edge(v, w) else 'blue'
                display(v, w, back_edge=True, msg=txt)
    print(count)
    return parent

while vertices:
    pivot = vertices.pop()
    parent = search(g_star, pivot)
    vertices = vertices - parent.keys()

1 2 blue
1 4 red
4 2 red <back edge>
4 3 red
3 2 blue <back edge>
10
5 9 blue
5 6 red
6 7 red
7 8 red
8 10 blue
10 9 blue <back edge>
12
11 12 red
12 13 red
13 14 red
14 15 blue
15 16 blue
10
17 18 red
17 19 blue
19 20 blue
20 18 red <back edge>
20 21 red
20 22 blue
22 23 blue
23 21 red <back edge>
16
24 26 blue
24 27 blue
24 28 red
24 29 red
29 30 red
26 25 blue
12


In [27]:
vertices = set(g_star.vertices)
stacked = set()
parent = dict()

def visit(graph, start):
    stacked.add(start)
    for w in graph.adjacent_to(start):
        if w not in parent:
            txt = 'red' if red.has_edge(start, w) else 'blue'
            parent[w] = start
            display(start, w, back_edge=False, msg=txt)
            visit(graph, w)
        elif w in stacked and w != parent[start]:
            txt = 'red' if red.has_edge(start, w) else 'blue'
            display(start, w, back_edge=True, msg=txt)
    
    stacked.remove(start)
    return parent


while vertices:
    pivot = vertices.pop()
    parent = {pivot : None}
    parent = visit(g_star, pivot)
    vertices = vertices - parent.keys()

1 2 blue
2 3 blue
3 4 red
4 1 red <back edge>
4 2 red <back edge>
5 9 blue
9 10 blue
10 8 blue
8 7 red
7 6 red
6 5 red <back edge>
11 12 red
12 13 red
13 14 red
14 15 blue
15 16 blue
17 18 red
18 20 red
20 19 blue
19 17 blue <back edge>
20 21 red
21 23 red
23 22 blue
22 20 blue <back edge>
24 26 blue
26 25 blue
24 27 blue
24 28 red
24 29 red
29 30 red
